### Add Imports

In [1]:
import numpy as np
import copy
import os
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


%matplotlib inline

%load_ext autoreload
%autoreload 2

### Read dataset and print the shapes

In [2]:
df = pd.read_csv('train.csv')

feature_names = df.columns
X = df.iloc[:, :-1].values
Y = df.iloc[:, -1].values

In [ ]:
#print("Feature Names: ")
#print(feature_names)
#print("Features Matrix (X):")
#print(X[:2])
#print("\nTarget Vector (Y):")
#print(Y[:2])

### See features

##### See Unique values per column

In [ ]:
def create_ranges(series, num_bins=20):
    min_val, max_val = series.min(), series.max()
    range_step = (max_val - min_val) / num_bins
    ranges = [f"{min_val + range_step * i:.2f} - {min_val + range_step * (i+1):.2f}" for i in range(num_bins)]
    return ranges

def print_ranges(df):
  for column in df.columns:
      unique_values = df[column].unique()

      if unique_values.size > 20 and np.issubdtype(df[column].dtype, np.number):
            print(f"Column '{column}' has more than 20 unique numeric values. Creating ranges:")
            print(create_ranges(df[column]))
      else:
        print(f"Unique values in '{column}':")
        print(unique_values)
      print()

#print_ranges(df)

##### Check for Nans

In [3]:
def replace_nans(df):
  df_numeric = df.select_dtypes(include=[np.number])
  df[df_numeric.columns] = df_numeric.fillna(0)
  return df

df = replace_nans(df)

### Feature Engineering

In [4]:
def preprocessDataset(df):
  numerical_data = df.select_dtypes(include=['int64', 'float64'])
  categorical_data = df.select_dtypes(include=['object'])

  # One hot encoding for categories
  categorical_data_encoded = pd.get_dummies(categorical_data)

  # Apply Min-Max Scaling to numerical data
  scaler = MinMaxScaler()
  numerical_data_scaled = scaler.fit_transform(numerical_data)
  numerical_data_scaled_df = pd.DataFrame(numerical_data_scaled, columns=numerical_data.columns)

  preprocessed_df = pd.concat([numerical_data_scaled_df, categorical_data_encoded], axis=1)
  preprocessed_df.drop('Id', axis=1, inplace=True)
  return preprocessed_df

preprocessed_df = preprocessDataset(df)
#print(preprocessed_df)

In [5]:
preprocessed_X = preprocessed_df.iloc[:, :-1].values
Y = preprocessed_df.iloc[:, -1].values

In [ ]:
print(preprocessed_X.shape)

(1460, 288)


#### Train

In [6]:
input_shape = (preprocessed_X.shape[1],)
model = Sequential([
    Dense(2024, input_shape=input_shape),
    Dense(1024),
    Dense(512),
    Dense(256),
    Dense(1)
])

model.compile(optimizer='adam',
              loss='mean_squared_error',
              metrics=[tf.keras.metrics.RootMeanSquaredError(name='rmse')])

In [7]:
model.fit(preprocessed_X, Y, epochs=100, batch_size=4)


Epoch 1/100
365/365 [==============================] - 23s 60ms/step - loss: 1.6227 - rmse: 1.2739
Epoch 2/100
365/365 [==============================] - 24s 64ms/step - loss: 0.0118 - rmse: 0.1085
Epoch 3/100
365/365 [==============================] - 22s 59ms/step - loss: 0.0090 - rmse: 0.0950
Epoch 4/100
365/365 [==============================] - 22s 60ms/step - loss: 0.0103 - rmse: 0.1016
Epoch 5/100
365/365 [==============================] - 22s 60ms/step - loss: 0.0191 - rmse: 0.1382
Epoch 6/100
365/365 [==============================] - 21s 58ms/step - loss: 0.0094 - rmse: 0.0969
Epoch 7/100
365/365 [==============================] - 22s 61ms/step - loss: 0.0104 - rmse: 0.1019
Epoch 8/100
365/365 [==============================] - 22s 60ms/step - loss: 2919.0339 - rmse: 54.0281
Epoch 9/100
365/365 [==============================] - 21s 59ms/step - loss: 68.1974 - rmse: 8.2582
Epoch 10/100
365/365 [==============================] - 22s 61ms/step - loss: 0.2868 - rmse: 0.5355
Epoc

# Predicton

In [8]:
# Read train data

test_df = pd.read_csv('test.csv')

ids = test_df['Id'].values.astype(int)
test_df = replace_nans(test_df)
preprocessed_test_df = preprocessDataset(test_df)



In [9]:
#print(preprocessed_test_df.shape)

unique_to_preprocessed_df = preprocessed_df.columns.difference(preprocessed_test_df.columns)
#print(unique_to_preprocessed_df)

# Populate difference of columns in test_Df
for column in unique_to_preprocessed_df:
    preprocessed_test_df[column] = 0

# maintain order of the columns too:
ordered_columns = [col for col in preprocessed_df.columns if col in preprocessed_test_df.columns]

preprocessed_test_df = preprocessed_test_df[ordered_columns]
preprocessed_test_df.drop('SalePrice', axis=1, inplace=True)

# check difference again:
unique_to_preprocessed_df = preprocessed_df.columns.difference(preprocessed_test_df.columns)
#print(unique_to_preprocessed_df)

#print("Shape of train: " + str(preprocessed_df.shape) + " Shape of test: " + str(preprocessed_test_df.shape))



<ipython-input-9-ec42950fbb20>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  preprocessed_test_df.drop('SalePrice', axis=1, inplace=True)


In [10]:
predictions = model.predict(preprocessed_test_df)

46/46 [==============================] - 1s 18ms/step


In [11]:
# concatenate predictions with ids and store to file.

result = np.column_stack((ids, predictions))

print("Predictions shape: " + str(predictions.shape))
print("ids shape: " + str(ids.shape))
print("result shape: " + str(result.shape))

Predictions shape: (1459, 1)
ids shape: (1459,)
result shape: (1459, 2)


In [12]:

result_df = pd.DataFrame(result, columns=['Id', 'SalePrice'])
result_df['Id'] = result_df['Id'].astype(int)


# Save the DataFrame to a CSV file
result_df.to_csv('predictions.csv', index=False)
